<font color='maroon' size=5A face="black body"> Get cleaned data </font>



In [1]:
%store -r dbcopy 
%store -r df_task2
%store -r user_behaviour

db = dbcopy

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans

<br/><font color='gold' size=7A face="black body"> Experience Analytics </font><br/><br/>
<p> <font color='maroon' size=6A face="black body"> Sub tasks perform the following tasks </font></p>
    <ul>
        <li>Aggregate, per customer, the following information (treat missing & outliers by replacing by the mean or the mode of the corresponding variable):
            <ul>
                <li>Average TCP retransmission</li>
                <li>Average RTT</li>
                <li>Handset type</li>
                <li>Average throughput</li>
            </ul>
        </li>
        <li>A Compute & list 10 of the top, bottom and most frequent:
            <ul>
                <li>TCP values in the dataset</li>
                <li>RTT values in the dataset.</li>
                <li>Throughput values in the dataset</li>
            </ul>
        </li>
        <li>Compute & report:
            <ul>
                <li>The distribution of the average throughput  per handset type and provide interpretation for your findings</li>
                <li>The average TCP retransmission view per handset type and provide interpretation for your findings</li>
            </ul>
        </li>
        <li>Using the experience metrics above, perform a k-means clustering (where k = 3) to segment users into groups of experiences and provide a brief description of each cluster. (The description must define each group based on your understanding of the data)</li>
    </ul>

In [10]:
new_netwok_df = dbcopy[['MSISDN/Number','Bearer Id','Handset Type','TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',\
                         'Avg RTT DL (ms)', 'Avg RTT UL (ms)',\
                         'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']]


In [11]:
new_netwok_df['Total TCP Retrans'] = new_netwok_df['TCP DL Retrans. Vol (Bytes)'] +\
    new_netwok_df['TCP UL Retrans. Vol (Bytes)']
new_netwok_df['Total Throughput'] = new_netwok_df['Avg Bearer TP DL (kbps)'] +\
    new_netwok_df['Avg Bearer TP DL (kbps)']

new_netwok_df['Total RTT'] = new_netwok_df['Avg RTT DL (ms)'] + new_netwok_df['Avg RTT UL (ms)']

new_netwok_df.head()

<ipython-input-11-85a76af12d99>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_netwok_df['Total TCP Retrans'] = new_netwok_df['TCP DL Retrans. Vol (Bytes)'] +\
<ipython-input-11-85a76af12d99>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_netwok_df['Total Throughput'] = new_netwok_df['Avg Bearer TP DL (kbps)'] +\
<ipython-input-11-85a76af12d99>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,MSISDN/Number,Bearer Id,Handset Type,TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Total TCP Retrans,Total Throughput,Total RTT
0,3.366496e+10,1.311448e+19,Samsung Galaxy A5 Sm-A520F,2.080991e+07,759658.664811,42.000000,5.000000,23.0,44.0,2.156957e+07,46.0,47.000000
1,3.368185e+10,1.311448e+19,Samsung Galaxy J5 (Sm-J530),2.080991e+07,759658.664811,65.000000,5.000000,16.0,26.0,2.156957e+07,32.0,70.000000
2,3.376063e+10,1.311448e+19,Samsung Galaxy A8 (2018),2.080991e+07,759658.664811,109.795706,17.662883,6.0,9.0,2.156957e+07,12.0,127.458589
3,3.375034e+10,1.311448e+19,undefined,2.080991e+07,759658.664811,109.795706,17.662883,44.0,44.0,2.156957e+07,88.0,127.458589
4,3.369980e+10,1.311448e+19,Samsung Sm-G390F,2.080991e+07,759658.664811,109.795706,17.662883,6.0,9.0,2.156957e+07,12.0,127.458589


In [12]:
aggregate = {'Total TCP Retrans':'sum', 'Total Throughput':'sum', 'Total RTT':'sum'}
columns = ['MSISDN/Number','Bearer Id','Handset Type', 'Total TCP Retrans', 'Total Throughput', 'Total RTT']
network_per_user_df = new_netwok_df[columns].groupby('MSISDN/Number').agg(aggregate).reset_index()
network_per_user_df.head()

,MSISDN/Number,Total TCP Retrans,Total Throughput,Total RTT
0,3.360100e+10,2.156957e+07,74.0,46.000000
1,3.360100e+10,2.156957e+07,96.0,31.000000
2,3.360100e+10,2.156957e+07,96.0,127.458589
3,3.360101e+10,7.607247e+05,408.0,84.000000
4,3.360101e+10,3.094040e+07,80790.0,119.000000


In [13]:

# top 5
network_per_user_df.sort_values(by='Total TCP Retrans', ascending=False)[:5]

,MSISDN/Number,Total TCP Retrans,Total Throughput,Total RTT
66616,3.366773e+10,8.354209e+09,624632.0,703.0
33988,3.365908e+10,7.941860e+09,328898.0,396.0
42210,3.366098e+10,7.374899e+09,416266.0,417.0
94334,3.376128e+10,6.043978e+09,198702.0,351.0
97859,3.376264e+10,5.879989e+09,906978.0,2860.0


In [14]:
# Bottom 5
network_per_user_df.sort_values(by='Total TCP Retrans', ascending=True)[:5]

,MSISDN/Number,Total TCP Retrans,Total Throughput,Total RTT
23369,3.365013e+10,97.0,60460.0,48.0
6326,3.361478e+10,128.0,16996.0,64.0
96625,3.376206e+10,129.0,63080.0,41.0
13487,3.362636e+10,134.0,20256.0,40.0
2162,3.360783e+10,143.0,32484.0,32.0


In [15]:
# most frequent
network_per_user_df['Total TCP Retrans'].value_counts().head(5)

2.156957e+07    47122
4.313915e+07     7468
6.470872e+07     1032
2.081121e+07      363
8.627829e+07      252
Name: Total TCP Retrans, dtype: int64